In [1]:
from simplenlg.framework import *
from simplenlg.lexicon import *
from simplenlg.realiser.english import *
from simplenlg.phrasespec import *
from simplenlg.features import *
import pandas as pd
import random
from sys import platform
import pyttsx3

In [2]:
from gtts import gTTS
  
# This module is imported so that we can 
# play the converted audio
import os
from playsound import playsound

import vlc
  
# The text that you want to convert to audio
good_answer = ["You are right Potter, but ", "Good Job Potter! ",
                        "Well done Potter, but ", "That's right Potter! ", "That's correct Potter! "]
  
# Language in which you want to convert
language = 'en'
  
# Passing the text and language to the engine, 
# here we have marked slow=False. Which tells 
# the module that the converted audio should 
# have a high speed
myobj = gTTS(text=good_answer[1], lang=language, slow=False)
  
# Saving the converted audio in a mp3 file named
# welcome 
myobj.save("welcome.mp3")
playsound("./welcome.mp3")

In [3]:
import spacy
from spacy.matcher import DependencyMatcher
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
matcher = DependencyMatcher(nlp.vocab)

pattern = [
    {
        "RIGHT_ID": "anchor_is",
        "RIGHT_ATTRS": {"ORTH": "is"}
    },
    {
        "LEFT_ID": "anchor_is",
        "REL_OP": ">",
        "RIGHT_ID": "is_subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "anchor_is",
        "REL_OP": ">",
        "RIGHT_ID": "is_attr",
        "RIGHT_ATTRS": {"DEP": "attr"},
    },
   
]

matcher.add("IS", [pattern])
doc = nlp("Unicorn horn isn't present")
matches = matcher(doc)
displacy.render(doc)
print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# Each token_id corresponds to one pattern dict
for idx in matches:
  match_id, token_ids = matches[matches.index(idx)]
  
  for i in range(len(token_ids)):
      print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

[]


In [4]:
# frase di benvenuto -> inizio esame
possible_sentences = ["Good morning Mr Potter.", "Welcome to the potions examination Mr Potter.", \
            "Good morning Mr Potter, I hope that you have studied this time.", "Good morning Potter, I hope you are better prepared than last time. "]
choose_sentence = list(random.sample(range(0, len(possible_sentences)), 1))        
print(choose_sentence)

[0]


In [5]:
## commenti in base alla penalità
good_answer = ["You are right Potter, but ", "Good Job Potter! ", "Well done Potter, but ", "That's right Potter! ", "That's correct Potter! "]
bad_answer = ["Pity, clearly fame isn't everything, is it, mr Potter?  ", "Nice try but of course you are wrong. ", "That's not correct Potter! ", "You don't know? Well let's try again..."]
indifferent_answer = ["Answer me in a meaningful way Potter! "]
neutral_answer = ["I see you confused Potter! "]

In [6]:
## risposta incompleta
lexicon = Lexicon.getDefaultLexicon()
realiser = Realiser(lexicon)
nlgFactory = NLGFactory(lexicon)
p = nlgFactory.createClause("the remaining ingredients")
verb = nlgFactory.createVerbPhrase("be")
verb.setPlural(True)
p.setVerbPhrase(verb)
p.setFeature(Feature.INTERROGATIVE_TYPE, InterrogativeType.WHAT_OBJECT)
phrase = realiser.realiseSentence(p)
print(phrase)

What are the remaining ingredients?


In [7]:
## frame non completo
                ## Are you sure that all the ingredients have been listed?
p = nlgFactory.createClause("you", "be sure")
p0 = nlgFactory.createClause("all the ingredients", "have been listed")
p.setComplement(p0)
p.setFeature(Feature.INTERROGATIVE_TYPE, InterrogativeType.YES_NO)
phrase = realiser.realiseSentence(p)
print(phrase)

Are you sure that all the ingredients has been listed?


In [8]:
def generate_tricky_question(cls, to_ask: str):
        """ Generate a tricky question with specified ingredient or not. 

        Args:
            to_ask (str): ingredient to ask
        
        Returns:
            str: generated question
        """
        lexicon = Lexicon.getDefaultLexicon()
        realiser = Realiser(lexicon)
        nlgFactory = NLGFactory(lexicon)

        if to_ask == "question_tricky": # question without specified ingredient
            random_indexes = list(random.sample(range(0, 2), 1))

            if random_indexes[0] == 0:
                ## Are you sure that all the ingredients have been listed?
                p = nlgFactory.createClause("you", "be sure")
                p0 = nlgFactory.createClause("all the ingredients", "have been listed")
                p.setComplement(p0)
                p.setFeature(Feature.INTERROGATIVE_TYPE, InterrogativeType.YES_NO)
            else:
                ## Are you sure you have mentioned all the ingredients?
                p = nlgFactory.createClause("you", "be sure")
                p0 = nlgFactory.createClause("you", "have mentioned", "all the ingredients")
                p.setComplement(p0)
                p.setFeature(Feature.INTERROGATIVE_TYPE, InterrogativeType.YES_NO)
        else: # question with specified ingredient
            ## Are you sure that INGNAME is an ingredient of this potion?
            p = nlgFactory.createClause("you", "be sure")
            p0 = nlgFactory.createClause(to_ask, "be", "this potion")
            p0.setIndirectObject("an ingredient of")
            p.setComplement(p0)
            p.setFeature(Feature.INTERROGATIVE_TYPE, InterrogativeType.YES_NO)
        
        phrase = realiser.realiseSentence(p)   
        
        return phrase

In [9]:
# chiede ingredienti pozione
#TODO
# aggiungere altre frasi
lexicon = Lexicon.getDefaultLexicon()
realiser = Realiser(lexicon)
nlgFactory = NLGFactory(lexicon)
p = nlgFactory.createClause("Can you tell me the ingredients of")
phrase = realiser.realiseSentence(p)
print(phrase)

Can you tell me the ingredients of.


In [10]:
from difflib import SequenceMatcher
SequenceMatcher(None, "Essenza", "Essenza di Cicuta").ratio()

0.5833333333333334

In [2]:
import spacy
from spacy.matcher import DependencyMatcher
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
#en_core_web_sm
#en_core_web_trf
matcher = DependencyMatcher(nlp.vocab)




negation = [[
    {
        "RIGHT_ID": "anchor_is",
        "RIGHT_ATTRS": {"ORTH": "is"}
    },
    {
        "LEFT_ID": "anchor_is",
        "REL_OP": ">",
        "RIGHT_ID": "is_neg",
        "RIGHT_ATTRS": {"DEP": "neg"},
    },
     
], 
[
    {
        "RIGHT_ID": "anchor_is",
        "RIGHT_ATTRS": {"ORTH": "are"}
    },
    
    {
        "LEFT_ID": "anchor_is",
        "REL_OP": ">",
        "RIGHT_ID": "is_neg",
        "RIGHT_ATTRS": {"DEP": "neg"},
    },
    
     
]]

ingredient =[
  
# [
#         {
#         "RIGHT_ID": "ingredient",
#         "RIGHT_ATTRS": {"POS": "NOUN"},

        
#     },
#     {
#         "LEFT_ID": "ingredient",
#         "REL_OP": ">",
#         "RIGHT_ID": "ingredient_Right",
#         "RIGHT_ATTRS": {"DEP": "compound"},
        
#     },
#      {
#         "LEFT_ID": "ingredient",
#         "REL_OP": ">",
#         "RIGHT_ID": "ingredient_RR",
#         "RIGHT_ATTRS": {"DEP": "amod"},
        
#     }],
    # [
    #     {
    #     "RIGHT_ID": "ingredient",
    #     "RIGHT_ATTRS": {"POS": "NOUN"},

        
    # },
    # {
    #     "LEFT_ID": "ingredient",
    #     "REL_OP": ">",
    #     "RIGHT_ID": "ingredient_Right",
    #     "RIGHT_ATTRS": {"DEP": "amod", "POS": "VERB"},
        
    # },
    #  {
    #     "LEFT_ID": "ingredient",
    #     "REL_OP": ">",
    #     "RIGHT_ID": "ingredient_RR",
    #     "RIGHT_ATTRS": {"DEP": "amod", "POS": "ADJ"},
        
    # }],
    #  [
    #     {
    #     "RIGHT_ID": "ingredient",
    #     "RIGHT_ATTRS": {"DEP" : "attr", "POS": "NOUN"},

        
    # },
    # {
    #     "LEFT_ID": "ingredient",
    #     "REL_OP": ">",
    #     "RIGHT_ID": "ingredient_Right",
    #     "RIGHT_ATTRS": {"POS": "ADJ"},
        
    # },
    #  {
    #     "LEFT_ID": "ingredient",
    #     "REL_OP": ">",
    #     "RIGHT_ID": "ingredient_RR",
    #     "RIGHT_ATTRS": { "DEP": "amod"},
        
    # }],

    [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "NOUN"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "compound"},
    }], 
    [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "VERB"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "compound"},
    }],
     [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "PROPN"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "compound"},
    }],
  [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": { "DEP": "attr"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "amod"},
    }], 
     [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "ADV", "DEP": "attr"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "compound"},
    }], 
    [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "ADJ", "DEP": "attr"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "advmod"},
    }], 
    [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": { "DEP": "acomp"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "amod"},
    }], 
     [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": { "DEP": "acomp"},

        
    },
    {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_Right",
        "RIGHT_ATTRS": {"DEP": "advmod"},
    }], 
    [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "NOUN", "DEP" : "nsubj"},


        
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient_",
        "RIGHT_ATTRS": {"DEP": "amod"},

        
    }],
    [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "NOUN", "DEP" : "nsubj"},


        
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient__",
        "RIGHT_ATTRS": {"DEP": "compound", "POS" : "ADJ"},

        
    }],
     [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "NOUN"},


        
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient__",
        "RIGHT_ATTRS": {"DEP": "amod"},

        
    }],
      [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"ORTH": "squill" },
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ".",
        "RIGHT_ID": "ingredient__",
        "RIGHT_ATTRS": {"ORTH": "bulb"},

        
    }
    ],
    
      [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "NOUN", "DEP" : "attr"},
        # "LEFT_ATTRS": {"POS": "NOUN"},


        
    }],
          [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "ADJ", "DEP" : "acomp"},
        # "LEFT_ATTRS": {"POS": "NOUN"},


        
    }],
              [
        {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "ADJ", "DEP" : "attr"},


        
    }],
     
    
     
     [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "VERB"},


        
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient__",
        "RIGHT_ATTRS": {"DEP": "nsubj"},

        
    }],
    
      [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "VERB"},


        
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient__",
        "RIGHT_ATTRS": {"DEP": "dobj"},

        
    }],
     [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "X"},


        
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient__",
        "RIGHT_ATTRS": {"DEP": "amod"},

        
    }],
      [
    {
      "RIGHT_ID": "ingredient",
      "RIGHT_ATTRS": { "POS": "PROPN" }
    },
    {
      "LEFT_ID": "ingredient",
      "REL_OP": ">",
      "RIGHT_ID": "ingredient_",
      "RIGHT_ATTRS": { "DEP": "amod" }
    }
  ],
    [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "VERB"},


        
    },
        {
        "LEFT_ID": "ingredient",
        "REL_OP": ">",
        "RIGHT_ID": "ingredient__",
        "RIGHT_ATTRS": {"DEP": "advmod"},

        
    }],
    
    [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "NOUN"},


        
    }],
    [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "PROPN"},


        
    }],
     [
         {
        "RIGHT_ID": "ingredient",
        "RIGHT_ATTRS": {"POS": "VERB"},  
    }],


    
    # Servirebbe per Moly ma matcherebe tutto
    #   [
    #      {
    #     "RIGHT_ID": "ingredient",
    #     "RIGHT_ATTRS": {"POS": "ADJ"},  
    # }]
   

]


import time
start_time = time.time()


    
matcher.add("IS_RIGHT", ingredient)
matcher.add("IS NOT", negation)

ing = "rose petals"
ing1 = "Moon water"

t =[ing,"the first ingredient is " + ing, ing +" is the first ingredient" ] 
test = "I forgot caterpillar"

doc = nlp(test)
matches = matcher(doc)

print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# Each token_id corresponds to one pattern dict
displacy.render(doc)
# for idx in matches:
#   match_id, token_ids = matches[matches.index(idx)]

#   for i in range(len(token_ids)):
#      print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

for match in matches: 
    match[1].sort()
print([(nlp.vocab.strings[match[0]], " ".join([doc[i].text for i in match[1]])) for match in matches])


[(15315539555477203417, [1, 0]), (15315539555477203417, [1, 2]), (15315539555477203417, [1])]


[('IS_RIGHT', 'I forgot'), ('IS_RIGHT', 'forgot caterpillar'), ('IS_RIGHT', 'forgot')]


In [22]:
import random
t = {0: 'Bezoar', 1: 'Unicorn Horn', 2: 'Pistletoe Berries'}
print(len(t)-1)
print(random.randint(0,len(t)-1))

2
0


In [105]:
# for i in t :
#     doc = nlp(i.lower())
#     matches = matcher(doc)

#     print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# # Each token_id corresponds to one pattern dict
#     displacy.render(doc)
# # for idx in matches:
# #   match_id, token_ids = matches[matches.index(idx)]
  
# #   for i in range(len(token_ids)):
# #      print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

#     for match in matches: 
#         match[1].sort()
#     print([(nlp.vocab.strings[match[0]], " ".join([doc[i].text for i in match[1]])) for match in matches])

In [52]:
import spacy
from spacy.matcher import DependencyMatcher
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
matcher = DependencyMatcher(nlp.vocab)

neg_ans = [[
    {
        "RIGHT_ID": "anchor_is",
        "RIGHT_ATTRS": {"ORTH": "is"}
    },
    {
        "LEFT_ID": "anchor_is",
        "REL_OP": ">",
        "RIGHT_ID": "is_neg",
        "RIGHT_ATTRS": {"DEP": "neg"},
    },
     
], 
[
    {
        "RIGHT_ID": "anchor_is",
        "RIGHT_ATTRS": {"ORTH": "are"}
    },
    
    {
        "LEFT_ID": "anchor_is",
        "REL_OP": ">",
        "RIGHT_ID": "is_neg",
        "RIGHT_ATTRS": {"DEP": "neg"},
    },
    
     
]]

yesOrNo = [[
    {
        "RIGHT_ID": "intJ",
        "RIGHT_ATTRS": {"POS": "INTJ", "ORTH": "yes"}
    }, 
     
],
[
    {
        "RIGHT_ID": "intJ",
        "RIGHT_ATTRS": {"POS": "INTJ", "ORTH": "no"}
    }, 
     
]]

    
matcher.add("INTJ", yesOrNo)
# matcher.add("NO", no)
# matcher.add("IS_LEFT", ingredient_L)
matcher.add("Negative Answer", neg_ans)



doc = nlp("yes, no, no , yes")
matches = matcher(doc)

print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# Each token_id corresponds to one pattern dict
displacy.render(doc)
for idx in matches:
  match_id, token_ids = matches[matches.index(idx)]
  
  #for i in range(len(token_ids)):
   #   print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

for match in matches: 
 match[1].sort()
[(nlp.vocab.strings[match[0]], " ".join([doc[i].text for i in match[1]])) for match in matches] 

[(91, [0]), (91, [6]), (91, [2]), (91, [4])]


[('INTJ', 'yes'), ('INTJ', 'yes'), ('INTJ', 'no'), ('INTJ', 'no')]

In [179]:
# import json

# with open("../Data/is_ingredient.json", "w") as fp:
#         json.dump(ingredient,fp)
# with open("../Data/is_not_ingredient.json", "w") as fp:
#         json.dump(negation,fp)
# with open("../Data/neg_ans.json", "w") as fp:
#         json.dump(neg_ans,fp)
# with open("../Data/yesOrNo.json", "w") as fp:
#         json.dump(yesOrNo,fp)


In [66]:
import pandas as pd
df = pd.read_csv("../Data/potions.csv", na_filter=False)
# df1.read_csv(path , )
df

,Polyjuice,Lacewing Flies,Bicorn Horn,Knotgrass,Fluxweed,Boomslang Skin,Hair
0,Felix Felicis,Ashwinder Egg,Squill Bulb,Murtlap Tentacle,Tincture Thyme,Occamy Eggshell,Powered Rue
1,Amortentia,Ashwinder Egg,Rose Thorn,Peppermint,Powered Moonstone,Pearl Dust,
2,Ageing Potion,Red Wine,Prune Juice,Tortoise Shell,Caterpillar,Bat Tongue,
3,Draught of Peace,Powdered Moonstone,Unicorn Horn,Porcupine Quills,,,
4,Wolfsbane Potion,Wolfsbane,,,,,
5,Skele-Gro,Chinese Chomping cabbage,Puffer Fish,Scarab Beetles,,,
6,Invisibility Potion,Cherries,,,,,
7,Elixir of Life,Sorcerer Stone,,,,,
8,Wiggenweld Potion,Wiggentree Bark,Moly,Dittany,Horklump Juice,Flobberworm Mucus,Chizpurfle Fangs
9,Pompion Potion,Flitterby Moth,Bouncing Bulb,Foxglove,,,


In [67]:
# df.rename(columns={"Polyjuice": "Potion"}, inplace=True)
df.rename(columns={"Polyjuice": "Potion"}, inplace=True)

df

,Potion,Lacewing Flies,Bicorn Horn,Knotgrass,Fluxweed,Boomslang Skin,Hair
0,Felix Felicis,Ashwinder Egg,Squill Bulb,Murtlap Tentacle,Tincture Thyme,Occamy Eggshell,Powered Rue
1,Amortentia,Ashwinder Egg,Rose Thorn,Peppermint,Powered Moonstone,Pearl Dust,
2,Ageing Potion,Red Wine,Prune Juice,Tortoise Shell,Caterpillar,Bat Tongue,
3,Draught of Peace,Powdered Moonstone,Unicorn Horn,Porcupine Quills,,,
4,Wolfsbane Potion,Wolfsbane,,,,,
5,Skele-Gro,Chinese Chomping cabbage,Puffer Fish,Scarab Beetles,,,
6,Invisibility Potion,Cherries,,,,,
7,Elixir of Life,Sorcerer Stone,,,,,
8,Wiggenweld Potion,Wiggentree Bark,Moly,Dittany,Horklump Juice,Flobberworm Mucus,Chizpurfle Fangs
9,Pompion Potion,Flitterby Moth,Bouncing Bulb,Foxglove,,,


In [68]:
for i in range(1, len(df.columns)):
    df.rename(columns={df.columns[i]:  str(i -1) }, inplace=True)
df

,Potion,0,1,2,3,4,5
0,Felix Felicis,Ashwinder Egg,Squill Bulb,Murtlap Tentacle,Tincture Thyme,Occamy Eggshell,Powered Rue
1,Amortentia,Ashwinder Egg,Rose Thorn,Peppermint,Powered Moonstone,Pearl Dust,
2,Ageing Potion,Red Wine,Prune Juice,Tortoise Shell,Caterpillar,Bat Tongue,
3,Draught of Peace,Powdered Moonstone,Unicorn Horn,Porcupine Quills,,,
4,Wolfsbane Potion,Wolfsbane,,,,,
5,Skele-Gro,Chinese Chomping cabbage,Puffer Fish,Scarab Beetles,,,
6,Invisibility Potion,Cherries,,,,,
7,Elixir of Life,Sorcerer Stone,,,,,
8,Wiggenweld Potion,Wiggentree Bark,Moly,Dittany,Horklump Juice,Flobberworm Mucus,Chizpurfle Fangs
9,Pompion Potion,Flitterby Moth,Bouncing Bulb,Foxglove,,,


In [72]:
df = df.set_index("Potion")
df
# df.index.name = None


,0,1,2,3,4,5
Potion,,,,,,
Felix Felicis,Ashwinder Egg,Squill Bulb,Murtlap Tentacle,Tincture Thyme,Occamy Eggshell,Powered Rue
Amortentia,Ashwinder Egg,Rose Thorn,Peppermint,Powered Moonstone,Pearl Dust,
Ageing Potion,Red Wine,Prune Juice,Tortoise Shell,Caterpillar,Bat Tongue,
Draught of Peace,Powdered Moonstone,Unicorn Horn,Porcupine Quills,,,
Wolfsbane Potion,Wolfsbane,,,,,
Skele-Gro,Chinese Chomping cabbage,Puffer Fish,Scarab Beetles,,,
Invisibility Potion,Cherries,,,,,
Elixir of Life,Sorcerer Stone,,,,,
Wiggenweld Potion,Wiggentree Bark,Moly,Dittany,Horklump Juice,Flobberworm Mucus,Chizpurfle Fangs


In [76]:
df.index[0]

'Felix Felicis'

In [77]:
df.to_json("test_potion.json")
# df1 = df.to_json

In [35]:
import pandas as pd
import numpy as np
df_potions= pd.read_json("test_potion.json")
df_potions =  df_potions.replace( np.nan, "", regex=True)
df_potions = df_potions.sample(n=3)
df_potions.index[0]


'Draught of Peace'

In [32]:
# df_potions = pd.read_json("Data/three_potions_test.json")
actual_frame = {}
response_frame = {}

# ----------------------------------------------------
# Version with quantities
# transform dataframe in list of dicts of ingredients

for item in df_potions[0].index.to_list():
        tmp = df_potions.loc[item].to_dict()
        for i in range(0,len(list(df_potions.loc["Felix Felicis"].to_dict().values()))):
            if tmp[i]=="":
                del tmp[i]
                
        actual_frame[item] = tmp


for item in actual_frame:
    # Construct responses dict
    tmp = {}
    for ingredient in actual_frame[item]:
        tmp[ingredient] = None
    response_frame[item] = tmp


print(actual_frame["Amortentia"])

{0: 'Ashwinder Egg', 1: 'Rose Thorn', 2: 'Peppermint', 3: 'Powered Moonstone', 4: 'Pearl Dust'}


In [11]:

print("--b1: 1--")
print("--b1: 2--")
print("--b2: 1--")
print("--b2: testing--")
print("--b1: testing--")

print("--b1: 1--")
print("--b1: 2--")
print("--b1: testing--")

print("--b1: 1--")
print("--b1: 3--")
print("--b1: testing--")












ciao  0
ciao  1
ciao  2
ciao  3
ciao  4
ciao  5
after recursive 4
t 4
after recursive 3
t 3
after recursive 2
t 2
after recursive 1
t 1
after recursive 0
t 0


In [11]:
t = ["first","second","third","fourth","fith", "ingredient", "I"]

k = "ashwinder egg"
for idx in t:
    print(idx in k)

False
False
False
False
False
False
False


In [7]:
i = [("Moon water","test")]
k = ("Moon water","test")
# print(k not in i)
print(i)
print(i.remove(k))
print(i)

[('Moon water', 'test')]
None
[]


{(C(i,p,propose,accept V reject)),
(C(i,p,propose,accept V reject))}